In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
sw = stopwords.words('turkish')
# https://github.com/otuncelli/turkish-stemmer-python
#!pip install TurkishStemmer
from TurkishStemmer import TurkishStemmer
stemmer = TurkishStemmer()


## 1- Read the dataset and parse it.

#     ------------------------------------------------------------------

## 2- For a CV (cv_listesi.xlsx), encode it using  count vectorizer,
##     then tfidf vectorizer,
##     then RNNs, document2vec  

#     ------------------------------------------------------------------

## 3- For a job ad (ilan_icerikleri.xlsx), encode it using  count vectorizer, 
##     then tfidf vectorizer,
##     then RNNs, document2vec 

#     ------------------------------------------------------------------


# ____________________________________
# Part1:

# _Read and Parse Data_

### First of all I will create a person class to make build an object for each person, therefore, it will be quite easier to work with this data. Then I will check some basics of data and then create one big text for each person which is consist of cleaned resume and cleaned skills of person. After all, our data frame will be two columns; ID and AllResume. I will be ready to feed this dataframe into CountVectorizer and TFIDF! 

### After these steps I will use the same methodology for jobs data!


In [2]:
CVS = pd.read_excel('data/cv_list.xlsx')

In [3]:
class Person:
    def __init__(self,ref,resume_all):
        self.ref = ref
        self.resume_all = resume_all

In [4]:
people = {}

In [5]:
len(CVS)

8012

In [6]:
for index in range(len(CVS)):
    person = CVS.iloc[index:index+1,:]
    
    Ref = person.RefNumber[index]
    Resume_all = person.ResumeAll[index]
    
    person_object = Person(Ref,Resume_all)
    people[Ref] = person_object
del CVS 

In [7]:
len(people) # all of the persons are turned into an object!

8012

In [8]:
example_person = people[13693]

In [9]:
example_person.resume_all

'<p>48 km 2x3 şeritli karayolu yapımı, toplam 2 x 10.080 m uzunluğunda 21 viyadük, toplam 1600 m uzunluğunda 2 çift tüp tünel, 9 adet kavşak, 15 alt ve üst geçit yapım işinde Kalite, Çevre ve İş Güvenliği Şefi olarak görev yaptım.</p> Office  Profosyonel, Link (Personel ve stok kontrol üniteleri), internet, MRP, ERP, Paket bazı özel programlar. <p>Yönetici olarak şirket için çalışmak, Yönetim Bilimleri, Kalite, İş Güvenliği, Çevre Yönetimi, Teknik ve İnsan Kaynakları bilgi, deneyim,\n</p><p>yetenek ve becerilerimi kullanmak.</p>,<p>Turizm / Tekirdağ\'da turizm sektörü için hediye malzemeleri üretmekte olan işyerinde Fabrika müdürü olarak çalıştım.\xa0</p><p><span style="letter-spacing: 0px">Benim üretim, malzeme planlama, insan kaynakları, kalite ve diğer idari işlemlerde yetki sahibyidim.</span><br></p> Office  Profosyonel, Link (Personel ve stok kontrol üniteleri), internet, MRP, ERP, Paket bazı özel programlar. <p>Yönetici olarak şirket için çalışmak, Yönetim Bilimleri, Kalite, İş G

#### Some of the data has extracted from webpages and it still includes tags, etc. To get rid of HTML code I will use a simple function which is below

In [10]:
def clean_some_char(sk):
    try:
        sk = sk.replace('İ','i')  # this letter comes from lowering the capital i of turkish : İ              
    except:
        pass
    try:
        sk = sk.replace('ı','i')                
    except:
        pass

    try:
        sk = sk.replace('ö','o')                
    except:
        pass

    try:
        sk = sk.replace('ü','u')
    except:
        pass

    try:
        sk = sk.replace('ş','s')
    except:
        pass

    try:
        sk = sk.replace('ç','c')
    except:
        pass
    
    try:
        sk = sk.replace('ğ','g')
    except:
        pass
    
    try:
        sk = sk.replace('\t','')
    except:
        pass
    
    try:
        sk = sk.replace('_','')
    except:
        pass
    
    return sk

In [11]:
# https://github.com/rasbt/python-machine-learning-book-2nd-edition/blob/master/code/ch08/ch08.ipynb 

def preprocessor(text): # I took this function from above and changed a bit 
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower())
    return text

#### I will use preprocessor and clean_some_char functions to clear all puncts etc. and turn the resume_alls into one big text with just words and nothing non-alphanumeric

#### Lets see how data will change after using our function on it 

In [12]:
#this cell takes some time
for person in people:
    people[person].resume_all = preprocessor(clean_some_char(people[person].resume_all))

In [13]:
example_person.resume_all

'48 km 2x3 seritli karayolu yapimi toplam 2 x 10 080 m uzunlugunda 21 viyaduk toplam 1600 m uzunlugunda 2 cift tup tunel 9 adet kavsak 15 alt ve ust gecit yapim isinde kalite çevre ve is guvenligi şefi olarak gorev yaptim office profosyonel link personel ve stok kontrol uniteleri internet mrp erp paket bazi ozel programlar yonetici olarak sirket icin calismak yonetim bilimleri kalite is guvenligi çevre yonetimi teknik ve insan kaynaklari bilgi deneyim yetenek ve becerilerimi kullanmak turizm tekirdag da turizm sektoru icin hediye malzemeleri uretmekte olan isyerinde fabrika muduru olarak calistim benim uretim malzeme planlama insan kaynaklari kalite ve diger idari islemlerde yetki sahibyidim office profosyonel link personel ve stok kontrol uniteleri internet mrp erp paket bazi ozel programlar yonetici olarak sirket icin calismak yonetim bilimleri kalite is guvenligi çevre yonetimi teknik ve insan kaynaklari bilgi deneyim yetenek ve becerilerimi kullanmak 400 m3 saat kapasiteli 3 adet b

## Finally let's check if we have any non-alphanumeric value on our resumes and skills

In [14]:
non_alpha_resume_all = []


for ref in people:
    person = people[ref]
    
    person.resume_all = person.resume_all.replace(' ','B') # trying to cancel the spaces. spaces are non-alphanum values.
    
    
    if len(person.resume_all) != 0:
        if person.resume_all.isalnum() != True:
            non_alpha_resume_all.append(ref)
    
   
    person.resume_all = person.resume_all.replace('B',' ')
    
    
non_alpha_resume_all

[]

In [15]:
for person in people:
    wp = []
    words = people[person].resume_all.split()
    for word in words:
        if stemmer.stem(word) not in sw:
            wp.append(stemmer.stem(word))
    people[person].resume_all =' '.join(wp)

In [16]:
example_person.resume_all

'48 km 2x3 serit karayo yapim topla 2 x 10 080 m uzunlug 21 viyaduk topla 1600 m uzunlug 2 cift tup tunel 9 adet kav 15 alt ust gecit yapim isin kali çevr is guvenlig şef olarak gorev yaptim offi profosyonel link personel stok kontrol unite internet mrp erp paket bazi ozel progra yonetiç olarak sirket iç calismak yone bilim kali is guvenlig çevr yonet teknik insan kaynaklari bilgi dene yetenek beceri kullanmak turizm tekirdag turizm sektor iç hedi malzeme uretmek olan isyer fabrika mudur olarak calis ben uret malzeme planla insan kaynaklari kali diger idari islem yetk sahiby offi profosyonel link personel stok kontrol unite internet mrp erp paket bazi ozel progra yonetiç olarak sirket iç calismak yone bilim kali is guvenlig çevr yonet teknik insan kaynaklari bilgi dene yetenek beceri kullanmak 400 m3 saat kapasite 3 adet beton santarli tum uret kali kontrol faaliyet yer getirmek hakedeis proje tesl iç gerek kali kontrol evraklarin hazirlamak turkmenistan standart enstitu tarfindan onay

In [17]:
# DATAFRAME CELL
refs = [ref for ref in people]
resume_alls= [people[ref].resume_all for ref in people]
df= pd.DataFrame()
df['Ref'] =refs
df['AllResume'] =resume_alls
df = df[df.AllResume != 'nan']

In [18]:
del people

# CountVec and Tfidf on Resumes

In [19]:
cvR = CountVectorizer(ngram_range=(1,2),dtype='int16') # to not lose the meanings I will use 1-2 n-grams. I wanted to use 3 as well but
                                                      # i am having MemmoryError

In [20]:
cv_allresume= cvR.fit_transform(list(df.AllResume))

In [21]:
cv_allresume.shape

(7767, 782900)

In [22]:
tfR = TfidfVectorizer(ngram_range=(1,2)) # to not lose the meanings I will use 1-2 n-grams. I wanted to use 3 as well but
                                        # i am having MemmoryError 

In [23]:
tf_allresume = tfR.fit_transform(list(df.AllResume))

In [24]:
tf_allresume.shape

(7767, 782900)

# Same procedure for Jobs Data

In [25]:
JOBS = pd.read_excel('data/job_ad_contents.xlsx')

In [26]:
class Job:
    def __init__(self,jobID,description):
        self.jobID = jobID
        self.description = description 

In [27]:
jobs={}

In [28]:
for index in range(len(JOBS)):
    job = JOBS.iloc[index:index+1,:]
    
    JobID = job.JobId[index]
    Description = job.Ilan[index]
    
    job_object = Job(JobID,Description)
    jobs[JobID] = job_object
del JOBS 

In [29]:
list(jobs.keys())[563]

2187737

In [30]:
example_job = jobs[2187737]

In [31]:
example_job.description

'Satış Danışmanı <strong>Satış Danışmanı - Ankara</strong><br><br>Mağazacılık Perakende satış, ev ve bahçe mobilyaları, ev tekstili ve dekorasyonu,\xa0ithalat, İhracat ve\xa0üretim\xa0konularında faaliyet gösteren firmamızın,\xa0<strong>Kervan</strong>\xa0ve\xa0<strong>Home Sweet Home</strong>\xa0 Ankara Mağazaları için\xa0<strong>"Satış Danışmanları"</strong>\xa0aranmaktadır.<br>\xa0<ul><li>Tercihen ev tekstili, mobilya, bahçe mobilyaları, aydınlatma veya dekorasyon ürünlerinin satışında tecrübesi olan,</li><li>Mağazacılık sektöründe satış veya kasa konularında en az\xa02 yıl deneyimli, </li><li>En az lise mezunu, tercihen yüksek okul veya üniversite mezunu, </li><li>Mağazacılığı meslek olarak benimsemiş kariyer yapmak isteyen, </li><li>Düzgün diksiyona sahip, güler yüzlü, aktif değişime ve gelişime açık olan, </li><li>İkna kabiliyeti yüksek iletişim becerileri güçlü, giyimine ve dış görünüşüne özen gösteren,</li><li>Ekip çalışmasına yatkın, İşinde fark yaratabilecek yetkinliklere sah

In [32]:
for job in jobs:
    jobs[job].description = preprocessor(clean_some_char(jobs[job].description)) 

In [33]:
example_job.description

'satis danismani satis danismani ankaramagazacilik perakende satis ev ve bahce mobilyalari ev tekstili ve dekorasyonu ithalat ihracat ve uretim konularinda faaliyet gosteren firmamizin kervan ve home sweet home ankara magazalari icin satis danismanlari aranmaktadir tercihen ev tekstili mobilya bahce mobilyalari aydinlatma veya dekorasyon urunlerinin satisinda tecrubesi olan magazacilik sektorunde satis veya kasa konularinda en az 2 yil deneyimli en az lise mezunu tercihen yuksek okul veya universite mezunu magazaciligi meslek olarak benimsemis kariyer yapmak isteyen duzgun diksiyona sahip guler yuzlu aktif degisime ve gelisime acik olan ikna kabiliyeti yuksek iletisim becerileri guclu giyimine ve dis gorunusune ozen gosteren ekip calismasina yatkin isinde fark yaratabilecek yetkinliklere sahip yogun is temposunda musteri odakli calisabilecek tercihen nebim v3 programi kullanabilen iş tanimimagazaya gelen musterileri karsilamak musteri ihtiyacini anlayip ilave alternatif urunler sunarak

In [34]:
for job in jobs:
    wp = []
    words = jobs[job].description.split()
    for word in words:
        if stemmer.stem(word) not in sw:
            wp.append(stemmer.stem(word))
    jobs[job].description =' '.join(wp)     
    

In [35]:
example_job.description

'satis danismani satis danismani ankaramagazacilik peraken satis ev bahce mobilyalari ev teksti dekorasyon ithalat ihracat uret konularinda faaliyet gosteren firmamiz kervan home sweet home ankar magazalari iç satis danismanlari aranmaktadir tercihen ev teksti mobilya bahce mobilyalari aydinlatm dekorasyon urun satisinda tecrube olan magazacilik sektorunde satis kas konularinda 2 yil deneyim lis mezun tercihen yuksek okul universi mezun magazacilig meslek olarak benimsemis kariyer yapmak isteyen duzg diksiyon sahip guler yuzl aktif degis gelis acik olan ikna kabi yuksek iletis beceri gucl giyim dis gorunusune ozen gosteren ekip calismasina yatki isin fark yaratabilecek yetkinlik sahip yog is tempo muster odakli calisabilecek tercihen nep v3 programi kullanabilen iş tanimimagaza gelen musteri karsilamak muster ihtiyacin anlayip ilave alternatif urunler sunarak satis yapmak magazani satis hedef gerek katki saglamak eksik urunler tespit yapip magaza yonetici bildirmek ürunler reyon duzen 

In [36]:
dff = pd.DataFrame()
dff["JobId"] = [jobs[job].jobID for job in jobs]
dff["Description"] = [jobs[job].description for job in jobs]
del jobs

In [37]:
cvJ = CountVectorizer(ngram_range= (1,2))

In [38]:
cv_jobs = cvJ.fit_transform(list(dff.Description))

In [39]:
cv_jobs.shape

(1000, 84673)

In [40]:
tfJ = TfidfVectorizer(ngram_range=(1,2))

In [41]:
tf_jobs = tfJ.fit_transform(list(dff.Description))

In [42]:
tf_jobs.shape

(1000, 84673)

In [43]:
df = df.reset_index().drop('index',axis=1)

In [44]:
df

,Ref,AllResume
0,473,mapco turkcellgroup bunye sirket ms backoffi s...
1,900,ail sirket insaat sektorunde pazarla satis des...
2,1323,bilgi isle yonetici nbsp mo tekstil roteks tek...
3,3748,setting up departments in the production facil...
4,6363,ic sati al faaliyet sorum sati al mudur olarak...
...,...,...
7762,122121260,idari is faaliyet servis temizlik guvenlik ara...
7763,122124775,microsoft offi word excel powerpoint
7764,122133442,elektrik tesisati elektronik devr dizayni elek...
7765,122141830,word paint excel microsoft offi araba parç mon...


In [45]:
dff = dff.reset_index().drop('index',axis=1)

In [46]:
dff

,JobId,Description
0,1870838,maka şofor 5 yil maka soforlug tecrube sahip d...
1,1676048,şofor suruç avrup bolge mudurlug bagli sube de...
2,2141993,şofor suruç sitil çizg şirket grup bunye cento...
3,2175311,şofor suruç alfred concierg hizmet turki ilk r...
4,2193892,şofor suruç yog is tempo esnek calisma saat uy...
...,...,...
995,2158805,operasyonel mukemmellik uzmani üniversite nbsp...
996,2041878,test ölcum muhendi 4 yillik makine muhendislig...
997,2195798,muster siparis tedarik uzman yardimci dunyani ...
998,2190412,android gelistiriç preferred qualifications nb...


In [47]:
dfff = pd.read_excel('data\job_ad_cv_evaluation_results_simplified.xlsx').drop('Unnamed: 0',axis=1)

In [48]:
dfff

,JobId,CvId,Label
0,490619,11702287,1
1,490619,100819267,1
2,490619,105843622,1
3,490619,106155661,1
4,490619,108241683,1
...,...,...,...
9909,2199387,5592918,0
9910,2199387,6450952,0
9911,2199387,6459447,0
9912,2199387,6485644,0


In [49]:
info_list=[]

In [50]:
counter = 0
for i in range(len(dfff)):
    label = int(dfff.iloc[i:i+1,:].Label)
    
    cv_ref = int(dfff.iloc[i:i+1,:].CvId)  
    cv_index = df[df.Ref == cv_ref].index
    
    

    job_id = int(dfff.iloc[i:i+1,:].JobId)    
    job_index = int(dff[dff.JobId == job_id].index[0])   
    
    
    
    try:
        cv_index = cv_index[0]
        info_list.append((cv_index,job_index,label))
    
        counter += 1 
        
    except:
        pass
counter

9549

In [51]:
del df,dff,dfff

In [52]:
dictionary = {"count_vec_cv": cv_allresume, "count_vec_job": cv_jobs, "tfidf_cv" : tf_allresume, "tfidf_jobs" : tf_jobs, 'applications' : info_list}

import pickle
import dbm
def create_database(filename, datas):
    db = dbm.open(filename, 'c')
    db["list"] = pickle.dumps(datas)
    
def read_from_db(filename):
    db = dbm.open(filename, 'r')
    m = pickle.loads(db["list"])
    return m

create_database("dictionary",dictionary)



In [53]:
stop

NameError: name 'stop' is not defined

In [ ]:
for i in info_list:
    cv = tf_allresume[i[0]].toarray()[0]
    job = tf_jobs[i[1]].toarray()[0]
    
    label = np.array([i[2]])
    merged = [*cv,*job,*label]
    dataframe.append(merged)
    